In [2]:
import pandas as pd
import numpy as np
import pickle

import gensim
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.test.utils import datapath
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize 


Idea: Match claims in reason dataset with IBM CoPA's by topic, then compare cosine similarities using avg W2V embeddings. Map topic to CoPA subject to embeddings, and one df to keep track of each topic in Reason.

In [3]:
pickle_copadict = open("./principle_argument_CoPA/PA_dict.pkl","rb")
principle_args = pickle.load(pickle_copadict)
print(principle_args)

{'Adolescent Rights': ['Many adolescents cannot make responsible decisions', 'Adolescents are as capable as adults'], 'Animal Rights': ['Animals should not be treated as property', 'There is nothing wrong with using animals to further human interests'], 'Big government': ['Public utility is best served by actions coordinated by central government', 'Public interest is best served and propelled by voluntary interactions,and not ones dictated by government'], 'Black market': ['Prohibiting products and activities makes them less visible and available, and thus less harmful', 'Prohibition is counterproductive and only leads to increased demand'], 'Clean energy': ['Humanity must embrace clean energy in order to fight climate change', 'Ecological concerns add further strain on the economy'], 'Coercion': ['A decisive and enforced policy is the best way to deliver a message', 'Enforcement tends to be less effective than persuasion and education'], 'Conservatism': ['The current system is workin

In [4]:
copa_df = pd.read_csv("./principle_argument_CoPA/IBM_Debater_(R)_CoPA-Motion-ACL-2019.v0.csv")
copa_df

,Action,Topic,Wikipedia title,Motion phrasing,Initial list?,# CoPAs,Adolescent rights,Animal rights,Big government,Black market,...,Right to privacy,Self-determination,Sexual morality,Subsidies,Technology,Tradition,Value of science,Virtual life,Wealth distribution,Welfare state
0,mandatory,Abstinence-only sex education,Abstinence-only sex education,Abstinence-only sex education should be mandatory,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0
1,end,affirmative action,Affirmative action,We should end affirmative action,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
2,increase,airport racial profiling in the United States,Airport racial profiling in the United States,We should increase airport racial profiling in...,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
3,ban,alcoholic beverages,Alcoholic drink,We should ban alcoholic beverages,1.0,4.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
4,abolish,the American Jobs Act,American Jobs Act,We should abolish the American Jobs Act,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pathlib
pathlib.Path().absolute()

PosixPath('/Users/joshy/cs577/577project-gitrepo/data')

In [7]:
try:
    reasons_df = pd.read_pickle('reasons_df.pkl')
except (OSError, IOError) as e:
    reasons_df = pd.read_csv("./reason-dataset.csv")
reasons_df.head()


,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,avg_w2v_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.5661276, 0.5389883, 0.6098048, 0.7101158, 0..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.44384283, 0.46450335, 0.62090343, 0.7182656..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.41737673, 0.5605264, 0.58117366, 0.53856754..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.40784317, 0.45864588, 0.5202222, 0.669624, ..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[0.46884528, 0.42420495, 0.49323082, 0.6081152..."


In [8]:
complete_df = pd.read_pickle('reasons_complete_df.pkl')
complete_df.head()

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,claim_SBERT_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.021533493, -0.16520593, 0.35402358, 0.43329..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.47159326, 0.4925952, -0.23349553, -0.087148..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.4615807, 0.68479395, 0.737707, 0.26567188, ..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.78423434, 0.56576806, 0.69618547, 0.0808582..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[-0.19426306, 0.5660061, -0.13704692, -0.07844..."


In [12]:
complete_df['avg_w2v_embedding'] = reasons_df['avg_w2v_embedding'].values
complete_df.head()
reasons_df=complete_df
# complete_df.to_pickle('reasons_complete_df.pkl') # contains BERT now

In [41]:
docs = reasons_df['line'].tolist()
docs[:2]

['And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.',
 'I believe that abortion cannot be justified because it is killing an innocent human being.']

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [35]:
docs[:2]

['And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.',
 'I believe that abortion cannot be justified because it is killing an innocent human being.']

In [36]:
reasons_embeddings = model.encode(docs)

In [43]:
# len(reasons_embeddings)
docs[:5]

['And if it is not possible for your to have a baby, due to shortage of money or time... Nowadays many couples would adopt unwanted babies.',
 'I believe that abortion cannot be justified because it is killing an innocent human being.',
 'Moreover, United Nations declaration says children have a right to protected before as well as after birth!',
 "Even if it doesn't have a brain, my belief is that the PERSON still has a soul. The soul is what gives the PERSON LIFE",
 "Can't you use a condom while having sex. Yes, having the baby puts the teen mother in a bad spot, but it's her fault for getting pregnant in the first place."]

In [44]:
# Sentence-BERT (SBERT) embeddings
reasons_df['claim_SBERT_embedding'] = reasons_embeddings

In [45]:
reasons_df

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,claim_SBERT_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.021533493, -0.16520593, 0.35402358, 0.43329..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.47159326, 0.4925952, -0.23349553, -0.087148..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.4615807, 0.68479395, 0.737707, 0.26567188, ..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.78423434, 0.56576806, 0.69618547, 0.0808582..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[-0.19426306, 0.5660061, -0.13704692, -0.07844..."
...,...,...,...,...,...,...,...,...,...
2843,2843,obama,Pro,p-quality,B64,"While I wish he would have accomplished more, ...","[217.3269, 203.28893, 110.26104, 295.20947, -1...","[0.5100023, 0.41690615, 0.553481, 0.7322623, 0...","[-0.4622768, 0.40663606, 0.124617755, 0.206470..."
2844,2844,obama,Pro,p-republicans,B64,Obama inherited many of the deficiencies lefto...,"[285.06503, 322.81372, 154.35544, 362.61212, -...","[0.39890993, 0.26361057, 0.3582708, 0.5272109,...","[-0.23139754, 0.6431857, -0.26683483, 0.696691..."
2845,2845,obama,Con,c-War,F38,He has gotten troops out of countries Bush...,"[166.51376, 182.42606, 281.36407, 244.29797, -...","[0.39314574, 0.4079126, 0.45908293, 0.6267522,...","[-0.0853933, 0.32471505, -0.16449334, 0.657798..."
2846,2846,obama,Pro,p-economy,K33,fincial reform- made the banks do their buince...,"[246.23344, 331.38638, 234.50464, 365.0967, -3...","[0.48969442, 0.34099227, 0.43148765, 0.6171639...","[-0.21821818, -0.08449653, 0.42121246, 0.39522..."


In [48]:
reasons_df.to_pickle('reasons_complete_df.pkl') # contains BERT now

In [52]:
CoPAs = pd.read_pickle(r'principle_argument_CoPA/PA_list.pkl')
model = SentenceTransformer('bert-base-nli-mean-tokens')
pa_embeddings = model.encode(CoPAs)

In [53]:
copa_embeds_tfidf_df = pd.DataFrame({'CoPAs': CoPAs, 'CoPA_SBERT_embedding': pa_embeddings})
copa_embeds_tfidf_df.to_pickle('./principle_argument_CoPA/PA_embeds_SBERT.pkl')

In [13]:
pd.read_pickle('./principle_argument_CoPA/PA_embeds_SBERT.pkl')

,CoPAs,CoPA_SBERT_embedding
0,Many adolescents cannot make responsible decis...,"[0.1932151, 0.0744904, 0.3792708, 0.472886, -0..."
1,Adolescents are as capable as adults,"[-0.43708387, -0.017553791, 0.9722701, 0.87072..."
2,Animals should not be treated as property,"[0.86525947, 0.64463997, 1.0701231, -0.2228895..."
3,There is nothing wrong with using animals to f...,"[0.8034599, 0.19940414, 0.31913838, -0.0333539..."
4,Public utility is best served by actions coord...,"[-0.44541442, 0.03217736, 0.88752425, 0.832821..."
...,...,...
69,"In the virtual world, people lose touch with r...","[0.094981305, 0.29126653, 2.0097678, 0.3957778..."
70,Society has a duty to minimize inequality by a...,"[-0.7280266, 0.19222933, 0.31404325, 0.5798784..."
71,The way to achieve a fair distribution of weal...,"[-0.48898932, -0.33286077, 0.618642, 0.7260264..."
72,The state has a duty to provide for the social...,"[-0.2321953, 0.034115504, 0.55941975, 0.337884..."


In [14]:
pd.read_pickle('reasons_complete_df.pkl') # contains BERT now

,id,topic,stance,substance,source,line,claim_tf_idf_embedding,tf_idf_pa_similarity,claim_SBERT_embedding,avg_w2v_embedding
0,0,abortion,Con,c-adopt,Q34,And if it is not possible for your to have a b...,"[227.53242, 95.16571, 90.81468, 413.58224, -44...","[0.48416767, 0.4241976, 0.54075974, 0.69915974...","[0.021533493, -0.16520593, 0.35402358, 0.43329...","[0.5661276, 0.5389883, 0.6098048, 0.7101158, 0..."
1,1,abortion,Con,c-kill,Q34,I believe that abortion cannot be justified be...,"[32.17483, 92.68718, 283.92633, 117.998245, -2...","[0.47884166, 0.43939564, 0.5063248, 0.70720005...","[0.47159326, 0.4925952, -0.23349553, -0.087148...","[0.44384283, 0.46450335, 0.62090343, 0.7182656..."
2,2,abortion,Con,c-baby_right,Q34,"Moreover, United Nations declaration says chil...","[-27.021324, -21.564283, 46.234787, 301.42996,...","[0.43071118, 0.45206025, 0.48450536, 0.5497553...","[0.4615807, 0.68479395, 0.737707, 0.26567188, ...","[0.41737673, 0.5605264, 0.58117366, 0.53856754..."
3,3,abortion,Con,c-baby_right,M25,"Even if it doesn't have a brain, my belief is ...","[357.5555, -13.070822, 262.6633, 333.12784, -2...","[0.44407952, 0.4239362, 0.49705744, 0.70402575...","[0.78423434, 0.56576806, 0.69618547, 0.0808582...","[0.40784317, 0.45864588, 0.5202222, 0.669624, ..."
4,4,abortion,Con,c-sex,O43,"Can't you use a condom while having sex. Yes, ...","[182.9457, 77.004105, 34.06093, 160.79187, -15...","[0.41161808, 0.39982748, 0.43759474, 0.6643834...","[-0.19426306, 0.5660061, -0.13704692, -0.07844...","[0.46884528, 0.42420495, 0.49323082, 0.6081152..."
...,...,...,...,...,...,...,...,...,...,...
2843,2843,obama,Pro,p-quality,B64,"While I wish he would have accomplished more, ...","[217.3269, 203.28893, 110.26104, 295.20947, -1...","[0.5100023, 0.41690615, 0.553481, 0.7322623, 0...","[-0.4622768, 0.40663606, 0.124617755, 0.206470...","[0.48527277, 0.50043356, 0.57313573, 0.6658959..."
2844,2844,obama,Pro,p-republicans,B64,Obama inherited many of the deficiencies lefto...,"[285.06503, 322.81372, 154.35544, 362.61212, -...","[0.39890993, 0.26361057, 0.3582708, 0.5272109,...","[-0.23139754, 0.6431857, -0.26683483, 0.696691...","[0.3314036, 0.3014497, 0.33637276, 0.43141872,..."
2845,2845,obama,Con,c-War,F38,He has gotten troops out of countries Bush...,"[166.51376, 182.42606, 281.36407, 244.29797, -...","[0.39314574, 0.4079126, 0.45908293, 0.6267522,...","[-0.0853933, 0.32471505, -0.16449334, 0.657798...","[0.41485694, 0.4361237, 0.46337822, 0.595646, ..."
2846,2846,obama,Pro,p-economy,K33,fincial reform- made the banks do their buince...,"[246.23344, 331.38638, 234.50464, 365.0967, -3...","[0.48969442, 0.34099227, 0.43148765, 0.6171639...","[-0.21821818, -0.08449653, 0.42121246, 0.39522...","[0.49895176, 0.36892983, 0.4601441, 0.55394155..."
